# 15 P2 Project

* ist1102903 Ana Alfaiate (33%)
  
* ist1102902 Maria Inês Trigueiro (33%)
  
* ist1102881 Raquel Coelho (33%)

Prof. Flávio Martins (and Prof. João Marques)

Lab Shift number: PB03

## PART I – Original Database Schema (SQLite)

##### E-R Diagram

![E-R Diagram](soccer.drawio.png "E-R Diagram")

#### 1. The Original Database Schema

Study the E-R Diagram above corresponding to the original database schema.


Lets download the SQLite database backup/dump!

In [1]:
!wget -c https://github.com/bdist/is-labs-data/raw/refs/heads/main/soccer/database.sqlite?download= -O /home/jovyan/data/database.sqlite

--2025-01-10 12:16:42--  https://github.com/bdist/is-labs-data/raw/refs/heads/main/soccer/database.sqlite?download=
140.82.121.3thub.com (github.com)... 
connected. to github.com (github.com)|140.82.121.3|:443... 
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/bdist/is-labs-data/refs/heads/main/soccer/database.sqlite?download=true [following]
--2025-01-10 12:16:43--  https://media.githubusercontent.com/media/bdist/is-labs-data/refs/heads/main/soccer/database.sqlite?download=true
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 416 Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



#### 2. SQL Queries

Consider the E-R Diagram above and write SQL queries for each of the following information needs.

In [1]:
%reload_ext sql
%config SqlMagic.displaycon = 0
%config SqlMagic.displaylimit = 100
%config SqlMagic.feedback = 0
%sql sqlite:////home/jovyan/data/database.sqlite --alias sqlite

1. What is the maximum number of goals recorded in a match? Output a single number.

In [3]:
%%sql sqlite
SELECT
  MAX(home_team_goal + away_team_goal) AS max_nr_goals_match
FROM
  MATCH;

max_nr_goals_match
12


2. List each league name and its number of teams sorted from high to low.

In [4]:
%%sql sqlite
SELECT
  l.name,
  COUNT(DISTINCT team_api_id) AS nr_teams
FROM
  league AS l,
  MATCH,
  team t
WHERE
  l.id = league_id
  AND (
    t.team_api_id = home_team_api_id
    OR t.team_api_id = away_team_api_id
  )
GROUP BY
  l.name
ORDER BY
  nr_teams desc

name,nr_teams
France Ligue 1,35
England Premier League,34
Spain LIGA BBVA,33
Italy Serie A,32
Germany 1. Bundesliga,30
Portugal Liga ZON Sagres,29
Netherlands Eredivisie,25
Belgium Jupiler League,24
Poland Ekstraklasa,22
Scotland Premier League,17


In [5]:
%%sql sqlite 
SELECT
  L.name,
  COUNT(M.team_api_id) AS nr_teams
FROM
  League AS L
  JOIN (
    SELECT
      league_id,
      home_team_api_id AS team_api_id
    FROM
      MATCH
    UNION
    SELECT
      league_id,
      away_team_api_id AS team_api_id
    FROM
      MATCH
  ) AS M ON L.id = M.league_id
GROUP BY
  L.name
ORDER BY
  nr_teams desc;

name,nr_teams
France Ligue 1,35
England Premier League,34
Spain LIGA BBVA,33
Italy Serie A,32
Germany 1. Bundesliga,30
Portugal Liga ZON Sagres,29
Netherlands Eredivisie,25
Belgium Jupiler League,24
Poland Ekstraklasa,22
Scotland Premier League,17


3. Who are the GoalKeepers (GK) that have played all the away games for their team? Output the `team_long_name` and `player_name` (assuming `match.away_player_1` is the goalkeeper column).

Division Unique with count

%%sql sqlite
-- ANSWER

SELECT
    T.team_long_name,
    P.player_name
FROM
    Match m
    JOIN Player P ON away_player_1 = player_api_id
    JOIN Team t ON away_team_api_id = t.team_api_id
GROUP BY
    away_team_api_id,
    away_player_1
HAVING 
    1 = (
        SELECT
            COUNT(DISTINCT COALESCE(away_player_1, 'NULL'))
        FROM
            MATCH
        WHERE 
            away_team_api_id = m.away_team_api_id
    );
-- division com o count (semelhante ao UNIQUE)

Division com o count

In [ ]:
%%sql sqlite
SELECT
  T.team_long_name,
  P.player_name
FROM
  MATCH m
  JOIN Player P ON away_player_1 = player_api_id
  JOIN team t ON away_team_api_id = T.team_api_id
GROUP BY
  away_team_api_id,
  away_player_1
HAVING
  COUNT(*) = (
    SELECT
      COUNT(*)
    FROM
      MATCH
    WHERE
      away_team_api_id = m.away_team_api_id
  );
--division com o count

Division com o EXCEPT

In [ ]:
%%sql sqlite
DROP INDEX IF EXISTS idx_team;

CREATE INDEX idx_team ON MATCH (away_team_api_id, away_player_1, match_api_id);

In [ ]:
%%sql sqlite
SELECT DISTINCT team_name, P.player_name goalkeeper
FROM Player P JOIN (
    
    SELECT T.team_long_name AS team_name, M.away_player_1
    FROM Team T JOIN MATCH M ON T.team_api_id = M.away_team_api_id
    WHERE NOT EXISTS (
        
        SELECT DISTINCT (match_api_id)
        FROM MATCH
        WHERE M.away_team_api_id = away_team_api_id
    
        EXCEPT
    
        SELECT DISTINCT(match_api_id)
        FROM MATCH
        WHERE T.team_api_id = away_team_api_id AND M.away_player_1 = away_player_1
      )
  
    ) AS LOL ON LOL.away_player_1 = P.player_api_id;

-- Division com o EXCEPT

First Iteration

In [4]:
%%sql sqlite
SELECT
  T.team_long_name,
  P.player_name
FROM
  (
    SELECT
      *
    FROM
      (
        SELECT
          away_team_api_id,
          away_player_1
        FROM
          MATCH
        GROUP BY
          away_team_api_id,
          away_player_1
      )
    GROUP BY
      away_team_api_id
    HAVING
      COUNT(*) = 1
  ) s 
  JOIN Player P ON s.away_player_1 = player_api_id
  JOIN team t ON s.away_team_api_id = T.team_api_id;

team_long_name,player_name
Fortuna Düsseldorf,Fabian Giefer
SV Darmstadt 98,Christian Mathenia
FC Energie Cottbus,Gerhard Tremmel
SC Paderborn 07,Lukas Kruse
RC Recreativo,Asier Riesgo
Watford,Heurelho Gomes
Dijon FCO,Baptiste Reynet
CD Tenerife,Sergio Aragones
DSC Arminia Bielefeld,Dennis Eilhoff


## PART II - New Database Schema (PostgreSQL)

#### 0. Migrate the SQLite Database to PostgreSQL

Instead of writing tedious migration code ourselves, we will be using the `pgloader` CLI tool to get started quickly. The migration task is setup using a `command-file`. Lets see inside that file.

In [3]:
!cat ./command-file.txt

load database
    from '/home/jovyan/data/database.sqlite'
    into postgresql://db:db@postgres/db

with include drop, create tables, create indexes, reset sequences,

prefetch rows = 100, max parallel create index = 1

set work_mem to '16MB', maintenance_work_mem to '512 MB';


The `pgloader` tool will take care of everything automatically including the foreign key constraints (which SQLite does not enforce by default). Lets run it.

In [4]:
!pgloader ./command-file.txt

2025-01-10T12:17:01.010642Z LOG pgloader version "3.6.10~devel"
2025-01-10T12:17:01.010642Z LOG Data errors in '/tmp/pgloader/'
2025-01-10T12:17:01.010642Z LOG Parsing commands from file #P"/home/jovyan/work/command-file.txt"
2025-01-10T12:17:01.127702Z LOG Migrating from #<SQLITE-CONNECTION sqlite:///home/jovyan/data/database.sqlite {1006A654E3}>
2025-01-10T12:17:01.127702Z LOG Migrating into #<PGSQL-CONNECTION pgsql://db@postgres:5432/db {1006A65663}>
2025-01-10T12:17:21.999456Z LOG report summary reset
             table name     errors       rows      bytes      total time
-----------------------  ---------  ---------  ---------  --------------
                  fetch          0          0                     0.000s
        fetch meta data          0         50                     0.074s
         Create Schemas          0          0                     0.000s
       Create SQL Types          0          0                     0.000s
          Create tables          0         14      

Lets take a closer look into the `match` table.

In [2]:
%load_ext sql
%config SqlMagic.displaycon = 0
%config SqlMagic.displaylimit = 100
%config SqlMagic.feedback = 0
%sql postgresql+psycopg://db:db@postgres/db --alias psql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [3]:
%reload_ext sql

In [10]:
%sqlcmd columns -t match

name,type,nullable,default,autoincrement,comment
id,BIGINT,False,nextval('match_id_seq'::regclass),True,None
country_id,BIGINT,True,None,False,None
league_id,BIGINT,True,None,False,None
season,TEXT,True,None,False,None
stage,BIGINT,True,None,False,None
date,TEXT,True,None,False,None
match_api_id,BIGINT,True,None,False,None
home_team_api_id,BIGINT,True,None,False,None
away_team_api_id,BIGINT,True,None,False,None
home_team_goal,BIGINT,True,None,False,None


Notice how `pgloader` even assigned the safest most-compatible data types for each column so that no data is lost in the migration.

##### 0.1. Introducing a limitation of the original database schema

The `match` table still does not look good to us... Can you try to write a SQL query for the following information need?

1. List all the teams that 'Cristiano Ronaldo' has played for (regardless of starting position)

In [8]:
%%sql psql
SELECT DISTINCT
  t.team_long_name
FROM
  MATCH m
  JOIN player p ON (
    p.player_api_id IN (
      m.home_player_1,
      m.home_player_2,
      m.home_player_3,
      m.home_player_4,
      m.home_player_5,
      m.home_player_6,
      m.home_player_7,
      m.home_player_8,
      m.home_player_9,
      m.home_player_10,
      m.home_player_11,
      m.away_player_1,
      m.away_player_2,
      m.away_player_3,
      m.away_player_4,
      m.away_player_5,
      m.away_player_6,
      m.away_player_7,
      m.away_player_8,
      m.away_player_9,
      m.away_player_10,
      m.away_player_11
    )
  )
  JOIN team t ON (
    (
      t.team_api_id = m.home_team_api_id
      AND p.player_api_id IN (
        m.home_player_1,
        m.home_player_2,
        m.home_player_3,
        m.home_player_4,
        m.home_player_5,
        m.home_player_6,
        m.home_player_7,
        m.home_player_8,
        m.home_player_9,
        m.home_player_10,
        m.home_player_11
      )
    )
    OR (
      t.team_api_id = m.away_team_api_id
      AND p.player_api_id IN (
        m.away_player_1,
        m.away_player_2,
        m.away_player_3,
        m.away_player_4,
        m.away_player_5,
        m.away_player_6,
        m.away_player_7,
        m.away_player_8,
        m.away_player_9,
        m.away_player_10,
        m.away_player_11
      )
    )
  )
WHERE
  p.player_name = 'Cristiano Ronaldo';

-- WILL NOT BE EVALUATED - We will be implementing a solution for this problem in the next section.

team_long_name
Manchester United
Real Madrid CF


Closing sqlite

Closing psql

#### 1. Functions, Stored Procedures, and Triggers

Provide the SQL instructions corresponding to each of the following tasks:

1. Create a new `match_player` table to store the data for the starting players (11) for each match (currently stored in the columns `match.home_player_DD` and `match.away_player_DD`). We are going to copy the data contained in the table `match` to `match_player` but we want to use a new column for the player position `position`.

In [4]:
%%sql psql
DROP TABLE IF EXISTS match_player;

CREATE TABLE
  match_player (
    match_id BIGINT,
    team_id BIGINT,
    player_id BIGINT, --- can be null
    POSITION INT,
    --type_team ENUM ('Away', 'Home'), --     type_team ENUM('Away', 'Home') adicionar OU fazer join com match
    PRIMARY KEY (match_id, team_id, POSITION), -- match_id, player_id (ideal) [not null ]
    FOREIGN KEY (match_id) REFERENCES MATCH (id),
    FOREIGN KEY (team_id) REFERENCES team (team_api_id),
    FOREIGN KEY (player_id) REFERENCES player (player_api_id)
  );

++
||
++
++

2. Populate the new table `match_player` using the data contained in the table `match` using a function or stored procedure.

In [5]:
%%sql psql
DELETE FROM match_player;

CREATE OR REPLACE PROCEDURE populate_match_player()
LANGUAGE plpgsql
AS $$
DECLARE
    match_row match%ROWTYPE;
    position INT;
    player_id INT;
BEGIN
    FOR match_row IN SELECT * FROM match LOOP
        -- HOME players
        FOR position IN 1..11 LOOP
            player_id := CASE position 
                WHEN 1 THEN match_row.home_player_1
                WHEN 2 THEN match_row.home_player_2
                WHEN 3 THEN match_row.home_player_3
                WHEN 4 THEN match_row.home_player_4
                WHEN 5 THEN match_row.home_player_5
                WHEN 6 THEN match_row.home_player_6
                WHEN 7 THEN match_row.home_player_7
                WHEN 8 THEN match_row.home_player_8
                WHEN 9 THEN match_row.home_player_9
                WHEN 10 THEN match_row.home_player_10
                WHEN 11 THEN match_row.home_player_11
            END;

            IF player_id IS NOT NULL THEN
                    EXECUTE format(
                        'INSERT INTO match_player (match_id, team_id, player_id, position)
                         VALUES (%s, %s, %s, %s)',
                        match_row.id,
                        match_row.home_team_api_id,
                        player_id,
                        position
                );
            END IF;
        END LOOP;

        -- Away players
        FOR position IN 1..11 LOOP
            player_id := CASE position
                WHEN 1 THEN match_row.away_player_1
                WHEN 2 THEN match_row.away_player_2
                WHEN 3 THEN match_row.away_player_3
                WHEN 4 THEN match_row.away_player_4
                WHEN 5 THEN match_row.away_player_5
                WHEN 6 THEN match_row.away_player_6
                WHEN 7 THEN match_row.away_player_7
                WHEN 8 THEN match_row.away_player_8
                WHEN 9 THEN match_row.away_player_9
                WHEN 10 THEN match_row.away_player_10
                WHEN 11 THEN match_row.away_player_11
            END;

            IF player_id IS NOT NULL THEN             

                EXECUTE format(
                    'INSERT INTO match_player (match_id, team_id, player_id, position)
                     VALUES (%s, %s, %s, %s)
                     ', 
                    match_row.id,
                    match_row.away_team_api_id,
                    player_id,
                    position
                );

            END IF;
        END LOOP;
    END LOOP;
END;
$$;

CALL populate_match_player(); -- player_id empty??

++
||
++
++

In [42]:
%%sql psql
SELECT
  *
FROM
  match_player

match_id,team_id,player_id,position
145,9996,39890,1
145,9996,38788,3
145,9996,38312,4
145,9996,26235,5
145,9996,26916,9
145,9996,94289,11
145,8635,34480,1
145,8635,38388,2
145,8635,26458,3
145,8635,13423,4


#### 2. SQL Queries

Consider the new `match_player` table above and write SQL queries for each of the following information needs.

1. List all the teams that 'Cristiano Ronaldo' has played for (regardless of starting position)

In [11]:
%%sql psql
SELECT DISTINCT
  (team_id),
  team_long_name
FROM
  match_player mp,
  player p,
  team t
WHERE
  mp.player_id = p.player_api_id
  AND p.player_name = 'Cristiano Ronaldo'
  AND t.team_api_id = mp.team_id;

team_id,team_long_name
8633,Real Madrid CF
10260,Manchester United


In [47]:
%%sql psql
SELECT DISTINCT
  T.team_long_name
FROM
  Team T
  JOIN match_player MP ON T.team_api_id = MP.team_id
  JOIN Player P ON MP.player_id = P.player_api_id
WHERE
  P.player_name = 'Cristiano Ronaldo';

team_long_name
Manchester United
Real Madrid CF


2. Who are the players that have played all the away games for their team? Output the `team_long_name` and `player_name`.

In [ ]:
-- +20 -30 

In [9]:
%%sql psql
    
DROP INDEX IF EXISTS idx_mp;
DROP INDEX IF EXISTS idx_m;

CREATE INDEX idx_mp ON match_player (team_id,  match_id, player_id); --player_id (not useful)
CREATE INDEX idx_m ON match (away_team_api_id);

++
||
++
++

In [10]:
%%sql psql
-- 
SELECT DISTINCT mp2.player_id
FROM match_player mp2 JOIN MATCH ON mp2.team_id = away_team_api_id
WHERE NOT EXISTS (
    SELECT DISTINCT match_id
    FROM match_player mp JOIN MATCH m ON mp.team_id = m.away_team_api_id
    WHERE mp2.team_id = mp.team_id
    EXCEPT
    SELECT DISTINCT match_id
    FROM match_player mp3 JOIN MATCH m2 ON mp3.team_id = m2.away_team_api_id
    WHERE mp2.team_id = mp3.team_id AND mp2.player_id = mp3.player_id);


KeyboardInterrupt



In [5]:
%%sql psql
SELECT
  t.team_long_name,
  p.player_name
FROM
  Player p
  JOIN (
    SELECT
      mp.player_id,
      mp.team_id
    FROM
      match_player mp JOIN MATCH ON mp.team_id = away_team_api_id
    GROUP BY
      mp.player_id,
      mp.team_id
    HAVING
      COUNT(DISTINCT mp.match_id) = (
        SELECT
          COUNT(DISTINCT m.match_id)
        FROM
          match_player m
        WHERE
          mp.team_id = m.team_id
        GROUP BY
          m.team_id
      )
  ) c ON p.player_api_id = c.player_id
  JOIN Team t ON c.team_id = t.team_api_id

UsageError: 'psql
SELECT' is not a valid connection identifier. Please pass the variable's name directly, as passing object attributes, dictionaries or lists won't work.
If you need help solving this issue, send us a message: https://ploomber.io/community


In [ ]:
-- division with count 1 s | except 1 min MELHOR MAIS CORRETA

In [12]:
%%sql psql
SELECT
  gp.player_id,
  player_name,
  t.team_long_name 
FROM
  (
    SELECT
      away_team_api_id,
      player_id,
      COUNT(player_id) AS games_player
    FROM
      MATCH,
      match_player mp
    WHERE
      mp.team_id = away_team_api_id
    GROUP BY
      away_team_api_id,
      player_id
    ORDER BY
      away_team_api_id
  ) gp
  JOIN (
    SELECT
      m.away_team_api_id,
      COUNT(*) AS total_games_team
    FROM
      MATCH m
    GROUP BY
      m.away_team_api_id
  ) gt ON gp.away_team_api_id = gt.away_team_api_id,
  player p,
  team t
WHERE
  gp.games_player = gt.total_games_team
  AND player_id = p.player_api_id
  AND gp.away_team_api_id = t.team_api_id

player_id,player_name,team_long_name
13390,Wojciech Skaba,Ruch Chorzów
105526,Adrian Mrowiec,Ruch Chorzów
459662,Volodymyr Tanchyk,Ruch Chorzów
555650,Lukasz Moneta,Ruch Chorzów
166660,Pieter Nys,Oud-Heverlee Leuven
641168,Din Sula,Oud-Heverlee Leuven
138155,Szymon Matuszek,Jagiellonia Białystok
206830,Bartlomiej Pawlowski,Jagiellonia Białystok
412056,Seweryn Michalski,Jagiellonia Białystok
429899,Tomasz Kowalski,Jagiellonia Białystok


In [18]:
%%sql psql
SELECT
  gp.player_id,
  player_name,
  team_long_name 
FROM
  (
    SELECT
      away_team_api_id,
      player_id,
      COUNT(away_team_api_id) AS games_player
    FROM
      MATCH,
      match_player mp
    WHERE
      mp.team_id = away_team_api_id
    GROUP BY
      away_team_api_id,
      player_id
    ORDER BY
      away_team_api_id
  ) gp
  JOIN (
    SELECT
      m.away_team_api_id,
      COUNT(*) AS total_games_team
    FROM
      MATCH m
    GROUP BY
      m.away_team_api_id
  ) gt ON gp.away_team_api_id = gt.away_team_api_id,
  player
WHERE
  gp.games_player = gt.total_games_team
  AND player_id = player.player_api_id
-- AND gp.away_team_api_id = t.team_api_id extra

player_id,player_name
13390,Wojciech Skaba
105526,Adrian Mrowiec
459662,Volodymyr Tanchyk
555650,Lukasz Moneta
166660,Pieter Nys
641168,Din Sula
138155,Szymon Matuszek
206830,Bartlomiej Pawlowski
412056,Seweryn Michalski
429899,Tomasz Kowalski


In [ ]:
count
1288

In [ ]:
%%sql psql
SELECT
  gp.player_id,
  player.player_name
FROM
  (
    SELECT
      mp.team_id AS away_team_api_id,
      mp.player_id,
      COUNT(mp.team_id) AS games_player
    FROM
      MATCH
      JOIN match_player mp ON mp.team_id = MATCH.away_team_api_id
    GROUP BY
      mp.team_id,
      mp.player_id
  ) gp
  JOIN (
    SELECT
      m.away_team_api_id,
      COUNT(*) AS total_games_team
    FROM
      MATCH m
    GROUP BY
      m.away_team_api_id
  ) gt ON gp.away_team_api_id = gt.away_team_api_id
  JOIN player ON gp.player_id = player.player_api_id
WHERE
  gp.games_player = gt.total_games_team;

3. What is the match with the highest number of goals? Output `team_long_name` for both home and away teams, and the total number of goals for each team.

In [58]:
%%sql psql
SELECT
  M.match_api_id AS MATCH,
  Th.team_long_name AS home_team,
  M.home_team_goal AS home_team_goals,
  Ta.team_long_name AS away_team,
  M.away_team_goal AS away_team_goals
FROM
  MATCH M
  JOIN Team Th ON M.home_team_api_id = Th.team_api_id
  JOIN Team Ta ON M.away_team_api_id = Ta.team_api_id
WHERE
  M.home_team_goal + M.away_team_goal >= (
    SELECT
      MAX(home_team_goal + away_team_goal)
    FROM
      MATCH
  );

match,home_team,home_team_goals,away_team,away_team_goals
820504,Motherwell,6,Hibernian,6
2030233,Real Madrid CF,10,Rayo Vallecano,2


#### 3. Query Optimization

Consider the new `player_attributes` table that contains the largest amount of rows.

1. Create the most appropriate index(es) for the following query: 

In [19]:
%%sql psql
DROP INDEX IF EXISTS player_att_index_c;

DROP INDEX IF EXISTS player_att_idx_hash;

DROP INDEX IF EXISTS player_att_bt;

++
||
++
++

In [11]:
%%sql psql
CREATE INDEX player_att_index_c ON player_attributes (attacking_work_rate, overall_rating)
--8 ms ANSWER makes more sense

++
||
++
++

In [17]:
%%sql psql
CREATE INDEX player_att_idx_hash ON player_attributes USING HASH (attacking_work_rate);

CREATE INDEX player_att_bt ON player_attributes (overall_rating);

-- another option

++
||
++
++

In [18]:
%%sql psql
EXPLAIN ANALYZE
SELECT
  MAX(overall_rating)
FROM
  player_attributes
WHERE
  attacking_work_rate = 'high';

max
94


In [18]:
%%sql psql
SELECT
  MAX(overall_rating)
FROM
  player_attributes
WHERE
  attacking_work_rate = 'high';

max
94


2. Create the most appropriate index(es) for the following query: 

In [42]:
%%sql psql
DROP INDEX IF EXISTS idx_id_foot;

DROP INDEX IF EXISTS idx_id_foot_filter;

DROP INDEX IF EXISTS idx_id_foot_filter2;

DROP INDEX IF EXISTS idx_id_foot_filter3;

DROP INDEX IF EXISTS idx_player_attributes_grouping;

DROP INDEX IF EXISTS idx_player_attributes_filtered;

++
||
++
++

In [32]:
%%sql psql
CREATE INDEX idx_id_foot ON player_attributes (player_api_id, preferred_foot);

-- index composto! btree
-- 68 ms

++
||
++
++

In [40]:
%%sql psql
CREATE INDEX idx_id_foot ON player_attributes (player_api_id, preferred_foot);

CREATE INDEX idx_id_foot_a ON player_attributes (crossing, potential);

-- index composto! btree
-- 66 ms ANSWER

++
||
++
++

In [23]:
%%sql psql
DROP INDEX IF EXISTS idx_player_attributes_grouping;

--CREATE INDEX idx_player_attributes_grouping ON player_attributes (crossing, potential, preferred_foot, player_api_id);
-- 84 ms

++
||
++
++

In [24]:
%%sql psql
--EXPLAIN (ANALYZE, BUFFERS) 
EXPLAIN ANALYZE
SELECT
  player_api_id,
  preferred_foot,
  MIN(crossing) AS min_crossing,
  MIN(potential) AS min_potential
FROM
  player_attributes
GROUP BY
  player_api_id,
  preferred_foot
HAVING
  MIN(crossing) = 80
  AND MIN(potential) >= 90
  -- 77 ms sem having
  -- 70 ms com having

QUERY PLAN
HashAggregate (cost=12398.58..12674.48 rows=31 width=29) (actual time=91.356..94.100 rows=2 loops=1)
"Group Key: player_api_id, preferred_foot"
Filter: ((min(crossing) = 80) AND (min(potential) >= 90))
Batches: 1 Memory Usage: 1809kB
Rows Removed by Filter: 12933
-> Seq Scan on player_attributes (cost=0.00..10559.29 rows=183929 width=29) (actual time=0.010..20.781 rows=183929 loops=1)
Planning Time: 0.255 ms
Execution Time: 94.363 ms


## PART III - The Web App (Flask + PostgreSQL)

#### 1. SQL View

1. Create a SQL View that lists the player names sorted by the most recent `player_attributes.date` first.

In [6]:
%%sql psql
--EXPLAIN ANALYZE 
DROP VIEW IF EXISTS players_index_view;

CREATE
OR REPLACE VIEW players_index_view AS
SELECT
  pa.player_api_id AS player_id,
  p.player_name,
  pa.date AS DATE
FROM
  player_attributes pa,
  player p
WHERE
  pa.player_api_id = p.player_api_id
  AND pa.date = (
    SELECT
      MAX(pa2.date)
      -->= ALL (Select pa2.date
    FROM
      player_attributes pa2
    WHERE
      pa2.player_fifa_api_id = pa.player_fifa_api_id
  )
  -- OR = SELECT MAX(pa2.date) FROM   player_attributes pa2      WHERE pa2.player_fifa_api_id = pa.player_fifa_api_id)
ORDER BY
  DATE DESC;

++
||
++
++

In [12]:
%%sql psql
--EXPLAIN ANALYZE 
DROP VIEW players_index_view;

CREATE
OR REPLACE VIEW players_index_view AS
SELECT
  player_id,
  p.player_name,
  DATE
FROM
  player p
  JOIN (
    SELECT
      pa.player_api_id AS player_id,
      MAX(pa.date) AS DATE
    FROM
      player_attributes pa
    GROUP BY
      pa.player_api_id
  ) ON player_id = p.player_api_id
ORDER BY
  DATE DESC;

/opt/conda/lib/python3.12/site-packages/sql/connection/connection.py:882: JupySQLRollbackPerformed: Current transaction is aborted. JupySQL executed a ROLLBACK operation.
  warnings.warn(
RuntimeError: (psycopg.errors.UndefinedTable) view "players_index_view" does not exist
[SQL: DROP VIEW players_index_view;]
(Background on this error at: https://sqlalche.me/e/20/f405)
If you need help solving this issue, send us a message: https://ploomber.io/community


In [105]:
%%sql psql
--extra query created by me
SELECT
  *
FROM
  players_index_view
  -- 79ms sem indices
  -- 81ms jogador, data
  -- 75ms data
  -- 79 ms data, jogador
  -- 212ms sem indices | 162
  -- 191 jogador data MORE SENSE
  -- 198 date
  -- 188 data jogador

player_id,player_name,date
231638,Dzikamai Andre Gwaze,2016-07-07 00:00:00
31432,Joe Hart,2016-06-23 00:00:00
25798,Bruno Lazaroni,2016-06-23 00:00:00
183414,David Stephens,2016-06-16 00:00:00
188058,Daniel Burn,2016-06-16 00:00:00
192625,Patrick Herrmann,2016-06-09 00:00:00
188540,Santiago Vergini,2016-06-09 00:00:00
178584,Sergiusz Prusak,2016-06-09 00:00:00
192319,Jonathan Mensah,2016-06-09 00:00:00
187150,Alejandro Martinuccio,2016-06-09 00:00:00


2. Create the most appropriate index(es) for the SQL view you just created. (Note: PostgreSQL does not have support for materialized views)

In [100]:
%%sql psql
DROP INDEX IF EXISTS idx_player_attributes_player_date
CREATE INDEX idx_player_attributes_player_date ON player_attributes (player_fifa_api_id, DATE DESC);
-- ANSWER

++
||
++
++

In [89]:
%%sql psql
DROP INDEX IF EXISTS idx_player_attributes_date
CREATE INDEX idx_player_attributes_date ON player_attributes (DATE DESC);

++
||
++
++

In [96]:
%%sql psql
CREATE INDEX idx_player_attributes_player_date ON player_attributes (DATE DESC, player_fifa_api_id);

-- 12 43
--CREATE INDEX idx_player_api_id ON player(player_api_id); -- este tambem??

++
||
++
++

In [103]:
%%sql psql
DROP INDEX IF EXISTS idx_player_api_id;

++
||
++
++

3. Please test your view using the following query.

In [5]:
%%sql psql
SELECT
  *
FROM
  players_index_view
ORDER BY
  DATE DESC
LIMIT
  20
  --WHERE player_api_id = 210065 ;

player_id,player_name,date
210065,Bebe,2016-07-07 00:00:00
45400,Ronnie Schwartz,2016-07-07 00:00:00
495841,Adalberto Penaranda,2016-07-07 00:00:00
184521,Florian Lejeune,2016-07-07 00:00:00
423024,Jhon Murillo,2016-07-07 00:00:00
317932,Baptiste Aloe,2016-07-07 00:00:00
307224,Kevin Koubemba,2016-07-07 00:00:00
512726,Yanis Mbombo Lokwa,2016-07-07 00:00:00
450002,Wallace,2016-07-07 00:00:00
167254,Jo-Gook Jung,2016-06-30 00:00:00


In [24]:
%%sql psql
SELECT
  pa.*,
  piv.player_name
FROM
  player_attributes pa
  JOIN players_index_view piv ON pa.player_api_id = piv.player_id
  AND pa.date = piv.date
WHERE
  player_api_id = 210065;

id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,player_name
19676,166074,210065,2016-07-07 00:00:00,72,75,right,medium,low,67,67,57,66,69,80,67,73,58,77,78,90,68,73,55,85,65,78,83,70,56,24,67,61,66,19,24,21,15,7,8,7,11,Bebe


In [29]:
%%sql psql
SELECT
  *
FROM
  players_index_view
LIMIT
  20;

player_id,player_name,date
210065,Bebe,2016-07-07 00:00:00
45400,Ronnie Schwartz,2016-07-07 00:00:00
495841,Adalberto Penaranda,2016-07-07 00:00:00
184521,Florian Lejeune,2016-07-07 00:00:00
423024,Jhon Murillo,2016-07-07 00:00:00
317932,Baptiste Aloe,2016-07-07 00:00:00
307224,Kevin Koubemba,2016-07-07 00:00:00
512726,Yanis Mbombo Lokwa,2016-07-07 00:00:00
450002,Wallace,2016-07-07 00:00:00
167254,Jo-Gook Jung,2016-06-30 00:00:00


#### 2. Create the players_index page on the Web App

Requirements:
- Should use the SQL View declared above
- Do not list more than 20 players.

1. Copy your players_index function from app.py to the cell below

```python
@app.route("/", methods=("GET",))
@app.route("/players", methods=("GET",))
@limiter.limit("1 per second")
def players_index():
    """Show all the accounts, most recent first."""

    with pool.connection() as conn:
        with conn.cursor() as cur:
            players = cur.execute(
                """
                SELECT player_id, player_name, date
                FROM players_index_view
                ORDER BY date DESC
                LIMIT 20;
                
                """,
                {},
            ).fetchall()
            log.debug(f"Found {cur.rowcount} rows.")

    return render_template("player/index.html", players=players)

```

2. Copy your final HTML template for this view onto the cell below

```html
{% extends "base.html" %}
{% block header %}
    <h1>
        {% block title %}
            Players
        {% endblock title %}
    </h1>
{% endblock header %}
{% block content %}
    {% for player in players %}
        <article class="post">
            <header>
                <div>
                    <h1>{{ player['player_name'] }}</h1>
                    <div class="about">Last Updated: {{ player['date'] }}</div>
                </div>
                <a class="action"
                   href="{{ url_for('player_update_view', player_api_id=player['player_id']) }}">Edit</a>
            </header>
        </article>
        {% if not loop.last %}<hr />{% endif %}
    {% endfor %}
{% endblock content %}

```

#### 3. Create the player_update_view and player_update_save routes on the Web App

Requirements:

- The app.route for player_update_view should take a player_api_id and show a nice editable HTML form.
             
- The app.route for player_update_save should be called on form submission and insert the new data into player_attributes. 

- On success, should redirect to the players_index

- On failure, should raise an appropriate exception with a error message

1. Copy your player_update_view function from app.py to the cell below

```python

@app.route("/players/<player_api_id>/update", methods=("GET",))
@limiter.limit("1 per second")
def player_update_view(player_api_id):
    """Show the page to update the account balance."""

    with pool.connection() as conn:
        with conn.cursor() as cur:
            player = cur.execute(
                """
                SELECT pa.*,  piv.player_name
                FROM player_attributes pa
                JOIN players_index_view piv
                ON pa.player_api_id = piv.player_id AND pa.date = piv.date
                WHERE player_api_id = %(player_api_id)s;
                """,
                {"player_api_id": int(player_api_id)},
            ).fetchone()
            log.debug(f"Found {cur.rowcount} rows.")
    if not player:
        raise ValueError("Player not found.")

    # At the end of the `connection()` context, the transaction is committed
    # or rolled back, and the connection returned to the pool.

#    return render_template("account/update.html", account=account)
    return render_template("player/update.html", player=player)
```

In [107]:
%%sql psql
SELECT
  column_name,
  data_type
FROM
  information_schema.columns
WHERE
  table_name = 'player_attributes';

column_name,data_type
gk_reflexes,bigint
player_fifa_api_id,bigint
player_api_id,bigint
gk_handling,bigint
gk_kicking,bigint
gk_positioning,bigint
id,bigint
overall_rating,bigint
potential,bigint
crossing,bigint


In [108]:
%%sql psql
SELECT
  COUNT(column_name)
FROM
  information_schema.columns
WHERE
  table_name = 'player_attributes';

count
42


2. Copy your final HTML template for this view onto the cell below

```python
{% extends "base.html" %}
{% block header %}
    <h1>
        {% block title %}
        Update Player Attributes for  {{ player['player_name'] }}
        {% endblock title %}
    </h1>
{% endblock header %}
{% block content %}
    <form method="post">

        <input type="hidden" name="player_api_id" value="{{ player['player_api_id'] }}">

        <label for="date">Last Update</label>
        <input type="datetime-local"
               id="date"
               name="date"
               value="{{ player['date'] or request.form['date']  }}" 
          />                <!-- player_attributes.date.strftime('%Y-%m-%dT%H:%M') -->
        <span>Previous Submission: {{ player['date'] }}</span>


        <label for="player_name">Player Name</label>
        <input name="player_name"
            id="player_name"
            type="text"
            value="{{ player['player_name'] }}"
            disabled />

        <label for="gk_reflexes">GK Reflexes</label>
        <input name="gk_reflexes"
            id="gk_reflexes"
            type="number"
            min="0"
            value="{{ request.form['gk_reflexes'] if request.form.get('gk_reflexes') else player['gk_reflexes'] }}"
             />
        <span>Previous Submission: {{ player['gk_reflexes'] }}</span>



        <label for="gk_handling">GK Handling</label>
        <input name="gk_handling"
            id="gk_handling"
            type="number"
            min="0"
            value="{{ request.form['gk_handling'] or player ['gk_handling'] }}"
            required />
            <span>Previous Submission: {{ player['gk_handling'] }}</span>


        <label for="gk_kicking">GK Kicking</label>
        <input name="gk_kicking"
                id="gk_kicking"
                type="number"
                value="{{ request.form['gk_kicking'] or player['gk_kicking'] }}"
                required />
                <span>Previous Submission: {{ player['gk_kicking'] }}</span>


        <label for="gk_positioning">GK Positioning</label>
        <input name="gk_positioning"
                id="gk_positioning"
                type="number"
                value="{{ request.form['gk_positioning'] or player['gk_positioning'] }}"
                required />
                <span>Previous Submission: {{ player['gk_positioning'] }}</span>


        <label for="overall_rating">Overall Rating</label>
        <input name="overall_rating"
            id="overall_rating"
            type="number"
            min="0"
            max="100"
            value="{{ request.form['overall_rating'] or player['overall_rating'] }}"
            required />
            <span>Previous Submission: {{ player['overall_rating'] }}</span>


        <label for="potential">Potential</label>
        <input name="potential"
            id="potential"
            type="number"
            min="0"
            max="100"
            value="{{ request.form['potential'] or player['potential'] }}"
            required />
            <span>Previous Submission: {{ player['potential'] }}</span>

        
        <label for="crossing">Crossing</label>
        <input name="crossing"
                id="crossing"
                type="number"
                value="{{ request.form['crossing'] or player['crossing'] }}"
                required />
                <span>Previous Submission: {{ player['crossing'] }}</span>


        <label for="finishing">Finishing</label>
        <input name="finishing"
                id="finishing"
                type="number"
                value="{{ request.form['finishing'] or player['finishing'] }}"
                required />
                <span>Previous Submission: {{ player['finishing'] }}</span>


        <label for="heading_accuracy">Heading Accuracy</label>
        <input name="heading_accuracy"
                id="heading_accuracy"
                type="number"
                value="{{ request.form['heading_accuracy'] or player['heading_accuracy'] }}"
                required />
                <span>Previous Submission: {{ player['heading_accuracy'] }}</span>


        <label for="short_passing">Short Passing</label>
        <input name="short_passing"
                id="short_passing"
                type="number"
                value="{{ request.form['short_passing'] or player['short_passing'] }}"
                required />
                <span>Previous Submission: {{ player['short_passing'] }}</span>


        <label for="volleys">Volleys</label>
        <input name="volleys"
                id="volleys"
                type="number"
                value="{{ request.form['volleys'] or player['volleys'] }}"
                required />
                <span>Previous Submission: {{ player['volleys'] }}</span>


        <label for="dribbling">Dribbling</label>
        <input name="dribbling"
                id="dribbling"
                type="number"
                value="{{ request.form['dribbling'] or player['dribbling'] }}"
                required />
                <span>Previous Submission: {{ player['dribbling'] }}</span>


        <label for="curve">Curve</label>
        <input name="curve"
                id="curve"
                type="number"
                value="{{ request.form['curve'] or player['curve'] }}"
                required />
                <span>Previous Submission: {{ player['curve'] }}</span>


        <label for="free_kick_accuracy">Free Kick Accuracy</label>
        <input name="free_kick_accuracy"
                id="free_kick_accuracy"
                type="number"
                value="{{ request.form['free_kick_accuracy'] or player['free_kick_accuracy'] }}"
                required />
                <span>Previous Submission: {{ player['free_kick_accuracy'] }}</span>


        <label for="long_passing">Long Passing</label>
        <input name="long_passing"
                id="long_passing"
                type="number"
                value="{{ request.form['long_passing'] or player['long_passing'] }}"
                required />
                <span>Previous Submission: {{ player['long_passing'] }}</span>


        <label for="ball_control">Ball Control</label>
        <input name="ball_control"
                id="ball_control"
                type="number"
                value="{{ request.form['ball_control'] or player['ball_control'] }}"
                required />
                <span>Previous Submission: {{ player['ball_control'] }}</span>


        <label for="acceleration">Acceleration</label>
        <input name="acceleration"
                id="acceleration"
                type="number"
                value="{{ request.form['acceleration'] or player['acceleration'] }}"
                required />
                <span>Previous Submission: {{ player['acceleration'] }}</span>


        <label for="sprint_speed">Sprint Speed</label>
        <input name="sprint_speed"
                id="sprint_speed"
                type="number"
                value="{{ request.form['sprint_speed'] or player['sprint_speed'] }}"
                required />
                <span>Previous Submission: {{ player['sprint_speed'] }}</span>


        <label for="agility">Agility</label>
        <input name="agility"
                id="agility"
                type="number"
                value="{{ request.form['agility'] or player['agility'] }}"
                required />
                <span>Previous Submission: {{ player['agility'] }}</span>


        <label for="reactions">Reactions</label>
        <input name="reactions"
                id="reactions"
                type="number"
                value="{{ request.form['reactions'] or player['reactions'] }}"
                required />
                <span>Previous Submission: {{ player['reactions'] }}</span>


        <label for="balance">Balance</label>
        <input name="balance"
                id="balance"
                type="number"
                value="{{ request.form['balance'] or player['balance'] }}"
                required />
                <span>Previous Submission: {{ player['balance'] }}</span>


        <label for="shot_power">Shot Power</label>
        <input name="shot_power"
                id="shot_power"
                type="number"
                value="{{ request.form['shot_power'] or player['shot_power'] }}"
                required />
                <span>Previous Submission: {{ player['shot_power'] }}</span>


        <label for="jumping">Jumping</label>
        <input name="jumping"
                id="jumping"
                type="number"
                value="{{ request.form['jumping'] or player['jumping'] }}"
                required />
                <span>Previous Submission: {{ player['jumping'] }}</span>


        <label for="stamina">Stamina</label>
        <input name="stamina"
                id="stamina"
                type="number"
                value="{{ request.form['stamina'] or player['stamina'] }}"
                required />
                <span>Previous Submission: {{ player['stamina'] }}</span>


        <label for="strength">Strength</label>
        <input name="strength"
                id="strength"
                type="number"
                value="{{ request.form['strength'] or player['strength'] }}"
                required />
                <span>Previous Submission: {{ player['strength'] }}</span>


        <label for="long_shots">Long Shots</label>
        <input name="long_shots"
                id="long_shots"
                type="number"
                value="{{ request.form['long_shots'] or player['long_shots'] }}"
                required />
                <span>Previous Submission: {{ player['long_shots'] }}</span>


        <label for="aggression">Aggression</label>
        <input name="aggression"
                id="aggression"
                type="number"
                value="{{ request.form['aggression'] or player['aggression'] }}"
                required />
                <span>Previous Submission: {{ player['aggression'] }}</span>


        <label for="interceptions">Interceptions</label>
        <input name="interceptions"
                id="interceptions"
                type="number"
                value="{{ request.form['interceptions'] or player['interceptions'] }}"
                required />
                <span>Previous Submission: {{ player['interceptions'] }}</span>


        <label for="positioning">Positioning</label>
        <input name="positioning"
                id="positioning"
                type="number"
                value="{{ request.form['positioning'] or player['positioning'] }}"
                required />
                <span>Previous Submission: {{ player['positioning'] }}</span>


        <label for="vision">Vision</label>
        <input name="vision"
                id="vision"
                type="number"
                value="{{ request.form['vision'] or player['vision'] }}"
                required />
                <span>Previous Submission: {{ player['vision'] }}</span>


        <label for="penalties">Penalties</label>
        <input name="penalties"
                id="penalties"
                type="number"
                value="{{ request.form['penalties'] or player['penalties'] }}"
                required />
                <span>Previous Submission: {{ player['penalties'] }}</span>


        <label for="marking">Marking</label>
        <input name="marking"
                id="marking"
                type="number"
                value="{{ request.form['marking'] or player['marking'] }}"
                required />
                <span>Previous Submission: {{ player['marking'] }}</span>


        <label for="standing_tackle">Standing Tackle</label>
        <input name="standing_tackle"
                id="standing_tackle"
                type="number"
                value="{{ request.form['standing_tackle'] or player['standing_tackle'] }}"
                required />
                <span>Previous Submission: {{ player['standing_tackle'] }}</span>


        <label for="sliding_tackle">Sliding Tackle</label>
        <input name="sliding_tackle"
                id="sliding_tackle"
                type="number"
                value="{{ request.form['sliding_tackle'] or player['sliding_tackle'] }}"
                required />
                <span>Previous Submission: {{ player['sliding_tackle'] }}</span>


        <label for="gk_diving">GK Diving</label>
        <input name="gk_diving"
                id="gk_diving"
                type="number"
                value="{{ request.form['gk_diving'] or player['gk_diving'] }}"
                required />
                <span>Previous Submission: {{ player['gk_diving'] }}</span>


        <label for="attacking_work_rate">Attacking Work Rate</label>
        <select name="attacking_work_rate" id="attacking_work_rate" required>
            <option value="">Select an option</option>
            <option value="Low" {% if (request.form['attacking_work_rate']|title) == 'Low' or (player['attacking_work_rate']|title) == 'Low' %}selected{% endif %}>Low</option>
            <option value="Medium" {% if (request.form['attacking_work_rate']|title) == 'Medium' or (player['attacking_work_rate']|title) == 'Medium' %}selected{% endif %}>Medium</option>
            <option value="High" {% if (request.form['attacking_work_rate']|title) == 'High' or (player['attacking_work_rate']|title) == 'High' %}selected{% endif %}>High</option>
        </select>
        <span>Previous Submission: {{ player['attacking_work_rate'] }}</span>


        <label for="defensive_work_rate">Defensive Work Rate</label>
        <select name="defensive_work_rate" id="defensive_work_rate" required>
            <option value="">Select an option</option>
            <option value="Low" {% if (request.form['defensive_work_rate']|title) == 'Low' or (player['defensive_work_rate']|title) == 'Low' %}selected{% endif %}>Low</option>
            <option value="Medium" {% if (request.form['defensive_work_rate']|title) == 'Medium' or (player['defensive_work_rate']|title) == 'Medium' %}selected{% endif %}>Medium</option>
            <option value="High" {% if (request.form['defensive_work_rate']|title) == 'High' or (player['defensive_work_rate']|title) == 'High' %}selected{% endif %}>High</option>
        </select>
        <span>Previous Submission: {{ player['defensive_work_rate'] }}</span>


        <label for="preferred_foot">Preferred Foot</label>
        <select name="preferred_foot" id="preferred_foot" required>
            <option value="Left" {% if (request.form['preferred_foot']|title) == 'Left' or (player['preferred_foot']|title) == 'Left' %}selected{% endif %}>Left</option>
            <option value="Right" {% if (request.form['preferred_foot']|title) == 'Right' or (player['preferred_foot']|title) == 'Right' %}selected{% endif %}>Right</option>
            <option value="" {% if request.form['preferred_foot'] == '' or player['preferred_foot'] is none %}selected{% endif %}>No Preference</option>
        </select>
        <span>Previous Submission: {{ player['preferred_foot'] }}</span>

        
        <input type="submit" value="Save" />
    </form>
    <hr />

{% endblock content %}
```

3. Copy your player_update_save function from app.py to the cell below

```python
@app.route("/players/<player_api_id>/update", methods=("POST",))
def player_update_save(player_api_id):
    """Update the player attributes."""

    try:
        # Extract the data from the form and only include fields that are not empty
        data = {}

        # List of fields to update (you can extend this list as needed)
        fields = [
            "gk_reflexes", "gk_handling", "gk_kicking", "gk_positioning",
            "overall_rating", "potential", "crossing", "finishing",
            "heading_accuracy", "short_passing", "volleys", "dribbling",
            "curve", "free_kick_accuracy", "long_passing", "ball_control",
            "acceleration", "sprint_speed", "agility", "reactions",
            "balance", "shot_power", "jumping", "stamina", "strength",
            "long_shots", "aggression", "interceptions", "positioning",
            "vision", "penalties", "marking", "standing_tackle", "sliding_tackle",
            "gk_diving", "attacking_work_rate", "defensive_work_rate",
            "preferred_foot", "date"
        ]

        for field in fields:
            value = request.form.get(field, None)
            if value is not None and value != "":
                data[field] = value
        
        data["player_api_id"] = player_api_id


        if "date" in data:
            try:
                converted_date = datetime.strptime(data["date"], '%Y-%m-%dT%H:%M')  # Converte para datetime
                data["date"] = converted_date
            except ValueError:
                raise ValueError("Invalid date format. Please use 'YYYY-MM-DDTHH:MM'.")

        with pool.connection() as conn:
            with conn.cursor() as cur:
                cur.execute("SELECT player_name FROM players_index_view WHERE player_id = %s", (player_api_id,))
                player_name = cur.fetchone()
                
                if player_name:
                    player_name = player_name[0]
                else:
                    raise ValueError("Player not found.")
                
        with pool.connection() as conn:
            with conn.cursor() as cur:
                set_clause = ", ".join([f"{key} = %s" for key in data.keys() if key != "player_api_id" ])
                log.debug(set_clause)    
                sql_query = f"""
                    UPDATE player_attributes
                    SET {set_clause}
                    WHERE player_api_id = %s
                """
                values = list(data.values())

                cur.execute(sql_query, values) 
                conn.commit()
                # The result of this statement is persisted immediately by the database
                # because the connection is in autocommit mode.

    # The connection is returned to the pool at the end of the `connection()` context but,
    # because it is not in a transaction state, no COMMIT is executed.
        flash(f"Player {player_name}'s attributes updated successfully!", "success")
        return redirect(url_for("players_index"))

    except ValueError as e:  # User-related error
        error_message = str(e)
        return render_template(
            "player/update.html",
            player_api_id=player_api_id,
            error_message=error_message,
        )
```